In [79]:
# import geopandas as gpd
# import fiona
# neighborhoods = gpd.read_file('neighborhoods.geojson').to_crs(fiona.crs.from_epsg(2263))
# neighborhoods
# boroughs = gpd.read_file('boroughs.geojson').to_crs(fiona.crs.from_epsg(2263))
# boroughs

In [158]:
def createIndex(shapefile):
    import rtree
    import fiona.crs
    import geopandas as gpd
    zones = gpd.read_file(shapefile).to_crs(fiona.crs.from_epsg(2263))
    index = rtree.Rtree()
    for idx,geometry in enumerate(zones.geometry):
        index.insert(idx, geometry.bounds)
    return (index, zones)

In [159]:
def findZone(p, index, zones):
    match = index.intersection((p.x, p.y, p.x, p.y))
    for idx in match:
        if zones.geometry[idx].contains(p):
            return idx
    return None

In [165]:
def processTrips(pid, records):
    import csv
    count =0
    import pyproj
    import shapely.geometry as geom
    import geopandas as gpd
    import fiona
    neighborhoods = gpd.read_file('neighborhoods.geojson').to_crs(fiona.crs.from_epsg(2263))
    boroughs = gpd.read_file('boroughs.geojson').to_crs(fiona.crs.from_epsg(2263))
    index_neighbourhood,zone_neighbourhood = createIndex('neighborhoods.geojson')
    index_boroughs,zone_boroughs = createIndex('boroughs.geojson')
    if pid ==0:
        next(records)
    for i in records:
        ls = i.split(',')
        lst=[]
        try:
            lst = [ls[3],ls[2],ls[5],ls[4]]#there is a blank line after header,this try catch handles it.
        except IndexError:
             pass
        if(lst and ('NULL' not in lst)):
            proj = pyproj.Proj(init="epsg:2263", preserve_units=True)    
            start = geom.Point(proj(float(lst[0]),float(lst[1])))
            end = geom.Point(proj(float(lst[2]),float(lst[3])))
            start_index = findZone(start,index_neighbourhood,zone_neighbourhood)
            end_index = findZone(end,index_boroughs,zone_boroughs)
            if None not in [start_index, end_index] :
                #print(count,boroughs['boroname'][end_index],neighborhoods['neighborhood'][start_index],int(1))
                yield(boroughs['boroname'][end_index],neighborhoods['neighborhood'][start_index],int(1))

In [166]:
rdd = sc.textFile('yellow1.csv')
counts = rdd.mapPartitionsWithIndex(processTrips)
a = counts.groupBy(lambda x:(x[0],x[1])).mapValues(len)

In [167]:
a.take(50)

[(('Manhattan', 'East Village'), 22046),
 (('Manhattan', 'Theater District'), 10858),
 (('Brooklyn', "Hell's Kitchen"), 761),
 (('Bronx', 'Upper East Side'), 319),
 (('Manhattan', 'Central Park'), 4411),
 (('Manhattan', 'West Village'), 16619),
 (('Manhattan', 'Midtown'), 49562),
 (('Manhattan', 'Kips Bay'), 9536),
 (('Manhattan', 'Nolita'), 2190),
 (('Manhattan', 'Chelsea'), 34550),
 (('Queens', 'Astoria'), 1160),
 (('Manhattan', "Hell's Kitchen"), 22045),
 (('Queens', 'Sunnyside'), 440),
 (('Manhattan', 'Murray Hill'), 8867),
 (('Manhattan', 'Morningside Heights'), 1872),
 (('Brooklyn', 'West Village'), 1325),
 (('Queens', 'Upper East Side'), 2221),
 (('Manhattan', 'Long Island City'), 802),
 (('Brooklyn', 'Chelsea'), 1849),
 (('Brooklyn', 'East Village'), 2369),
 (('Bronx', 'Gramercy'), 27),
 (('Manhattan', 'Flatiron District'), 5921),
 (('Manhattan', 'Battery Park City'), 2594),
 (('Queens', 'LaGuardia Airport'), 983),
 (('Brooklyn', 'NoHo'), 390),
 (('Brooklyn', 'Chinatown'), 380)

In [168]:
from heapq import nlargest
b=a.map(lambda x:(x[0][0],x[0][1],x[1]))
result = b.groupBy(lambda x: x[0]).flatMap(lambda g: nlargest(3, g[1], key=lambda x: x[2]))
result.saveAsTextFile("output1234")

In [13]:
#print(result)

In [10]:
# if __name__=='__main__':
#     rdd = sc.textFile('yellow.csv')
#     counts = rdd.mapPartitionsWithIndex(processTrips)
#     a = counts.groupBy(lambda x:(x[0],x[1])).mapValues(len)
#     from heapq import nlargest
#     b=a.map(lambda x:(x[0][0],x[0][1],x[1]))
#     result = b.groupBy(lambda x: x[0]).flatMap(lambda g: nlargest(3, g[1], key=lambda x: x[2])).collect()
#     print(result)
    

[('Manhattan', 'Midtown', 101), ('Manhattan', 'East Village', 81), ('Manhattan', 'Upper East Side', 73), ('Brooklyn', 'Williamsburg', 20), ('Brooklyn', 'West Village', 8), ('Brooklyn', 'Lower East Side', 8), ('Bronx', 'Upper East Side', 4), ('Bronx', 'Midtown', 3), ('Bronx', 'Chelsea', 2), ('Queens', 'Midtown', 7), ('Queens', 'Astoria', 5), ('Queens', 'Williamsburg', 4)]
